<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spacy_matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U spacy
!python -m spacy download zh_core_web_lg

     |████████████████████████████████| 6.0 MB 4.9 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 10.1 MB 41.3 MB/s 
     |████████████████████████████████| 628 kB 53.7 MB/s 
     |████████████████████████████████| 451 kB 51.6 MB/s 
     |████████████████████████████████| 181 kB 51.9 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 603.9 

In [13]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Doc, Span, Token
from spacy import displacy

In [7]:
markets = ['體育用品市場']
CHECKPOINT = 'zh_core_web_lg'

In [8]:
nlp = spacy.load(CHECKPOINT)

nlp.tokenizer.initialize(pkuseg_model="mixed")
nlp.disable_pipe('ner')

sentence = '體育用品市場'
doc = nlp(sentence)
for token in doc:
    print(token.text,token.pos_)

Downloading: "https://github.com/lancopku/pkuseg-python/releases/download/v0.0.16/mixed.zip" to /root/.pkuseg/mixed.zip
100%|██████████| 47330222/47330222 [00:00<00:00, 101969582.05it/s]


體育 NOUN
用品 NOUN
市場 NOUN


In [15]:
market_patterns = [[{"TEXT": market}] for market in markets]

def label_market(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    token = doc[start:end]
    next_token = doc[end:end+1]
    new_ents = []
    for ent in doc.ents:
        # if token.text not in ent.text:
            new_ents.append(ent)
    new_ent = Span(doc, start, end, label="FIN-MARKET")
    new_ents.append(new_ent)
    doc.ents = new_ents

nlp = spacy.load(CHECKPOINT)
user_dict = markets

matcher = Matcher(nlp.vocab)
matcher.add('market',market_patterns,on_match=label_market)

nlp.tokenizer.initialize(pkuseg_model="mixed")
nlp.tokenizer.pkuseg_update_user_dict(user_dict)
nlp.disable_pipe('ner')

sentence = '體育用品市場'
doc = nlp(sentence)
matches = matcher(doc)

displacy.render(doc, jupyter=True, style="ent")
